#### Hybrid Retriever-Combining Dense And Sparse Retriever

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

In [2]:
docs = [
    Document(page_content="Sparse retrieval methods like BM25 rely on keyword matching."),
    Document(page_content="Dense retrieval uses embeddings to capture semantic similarity between texts."),
    Document(page_content="Hybrid retrieval combines sparse and dense methods for better accuracy."),
    Document(page_content="A vector database stores embeddings for fast similarity search."),
    Document(page_content="ElasticSearch is a popular tool for keyword-based search."),
    Document(page_content="Pinecone and Weaviate are commonly used for dense vector retrieval.")]

In [3]:
embedding_model = HuggingFaceEmbeddings(model='all-miniLM-L6-v2')
dense_vectorstore = FAISS.from_documents(docs,embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

/home/bhupen/Python_Learning/RAG/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## Creating Sparse Retriever(BM25)
sparse_retriever = BM25Retriever.from_documents(docs)
sparse_retriever.k=3

## combine with Ensemble Retriever
hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever,sparse_retriever],
    weights=[0.7,0.3]
)

In [7]:
query = "What are the Sparse retrieval methods?"
results = hybrid_retriever.invoke(query)

for i,doc in enumerate(results):
    print(f"\n Document {i+1}:\n{doc.page_content}")


 Document 1:
Sparse retrieval methods like BM25 rely on keyword matching.

 Document 2:
Pinecone and Weaviate are commonly used for dense vector retrieval.

 Document 3:
Hybrid retrieval combines sparse and dense methods for better accuracy.

 Document 4:
Dense retrieval uses embeddings to capture semantic similarity between texts.

 Document 5:
ElasticSearch is a popular tool for keyword-based search.


##### RAG pipeline with hybrid retriever

In [9]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [10]:
prompt = PromptTemplate.from_template("""
Answer the question based on the context below.
                                      
                                      Context:
                                      {context}
                                      
                                      Question:{input}
                                      """)

llm = init_chat_model("openai:gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7fa6e0114c20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fa6e01156a0>, root_client=<openai.OpenAI object at 0x7fa6e1b62270>, root_async_client=<openai.AsyncOpenAI object at 0x7fa6e0115400>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [11]:
document_chain = create_stuff_documents_chain(llm=llm,prompt=prompt)

rag_chain = create_retrieval_chain(retriever=hybrid_retriever,combine_docs_chain=document_chain)

rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fa6fd93c590>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7fa6fd93d7f0>, k=3)], weights=[0.7, 0.3]), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context below.\n\n                                      Context:\n                                     

In [12]:
query = {"input":"What methods i can use to create sparse retrieval?"}
response = rag_chain.invoke(query)

print('Answer:',response['answer'])

print(f"Source Documents:")
for i,doc in enumerate(response['context']):
    print(f"\nDoc {i+1}: {doc.page_content}")

Answer: To create sparse retrieval, you can use methods such as BM25, which rely on keyword matching. Other techniques could include TF-IDF (Term Frequency-Inverse Document Frequency) or other boolean retrieval methods that focus on the presence or absence of specific keywords in the text.
Source Documents:

Doc 1: Hybrid retrieval combines sparse and dense methods for better accuracy.

Doc 2: Sparse retrieval methods like BM25 rely on keyword matching.

Doc 3: Dense retrieval uses embeddings to capture semantic similarity between texts.

Doc 4: A vector database stores embeddings for fast similarity search.


In [13]:
query = {"input":"Popular tool for keyword-based search?"}
response = rag_chain.invoke(query)

print('Answer:',response['answer'])

print(f"Source Documents:")
for i,doc in enumerate(response['context']):
    print(f"\nDoc {i+1}: {doc.page_content}")

Answer: The popular tool for keyword-based search is ElasticSearch.
Source Documents:

Doc 1: ElasticSearch is a popular tool for keyword-based search.

Doc 2: Sparse retrieval methods like BM25 rely on keyword matching.

Doc 3: Hybrid retrieval combines sparse and dense methods for better accuracy.

Doc 4: Dense retrieval uses embeddings to capture semantic similarity between texts.

Doc 5: A vector database stores embeddings for fast similarity search.

Doc 6: Pinecone and Weaviate are commonly used for dense vector retrieval.
